In [1]:
import polars as pl
import duckdb

In [ ]:
lf = pl.scan_csv(
    "../Datasets/VCF_annovar.vcf",
    skip_rows=444,
    separator="\t",
    schema_overrides={"#CHROM": pl.Utf8},
)

In [3]:
# df.limit(5).collect()

In [4]:
# df.select(pl.col("#CHROM").alias("CHROM").str.replace("chr", ""), ).limit(5).collect()

In [5]:
# df.select(
#     pl.col("#CHROM").alias("CHROM").str.replace("chr", ""),
#     pl.col(["POS", "REF", "ALT"]),
# ).limit(5).collect()

In [6]:
# df.select(
#     pl.col("#CHROM").alias("CHROM").str.replace("chr", ""),
#     pl.col(["POS", "REF", "ALT"]),
#     pl.col(["INFO"]).str.extract(r"AC=(\d+)").cast(pl.Int32).alias("AC"),
#     pl.col(["INFO"])
# ).limit(5).collect()

In [7]:
# lf = df.select(
#     pl.col("#CHROM").alias("CHROM").str.replace("chr", ""),
#     pl.col(["POS", "REF", "ALT"]),
#     pl.col(["INFO"])
# )

In [8]:
# Supposons que la colonne qui contient les infos s'appelle 'info'
def parse_info(row):
    pairs = row.split(";")
    return {k: v for k, v in (pair.split("=", 1) for pair in pairs if "=" in pair)}


In [9]:
# Transformer la colonne info
lf_parsed = (
    lf.with_columns(
        pl.col("INFO").map_elements(parse_info, return_dtype=pl.Struct).alias("parsed")
    )
    .unnest("parsed")  # Déplie le dictionnaire en colonnes
    # .drop("INFO")  # Optionnel : supprimer la colonne originale
)


In [10]:
lf_parsed.limit(5).collect()

#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,1BD4I,1BD4M,1BD4P,AC,AF,AN,BaseQRankSum,DP,ExcessHet,FS,MLEAC,MLEAF,MQ,MQRankSum,QD,ReadPosRankSum,SOR,ANNOVAR_DATE,Func.refGeneWithVer,Gene.refGeneWithVer,GeneDetail.refGeneWithVer,ExonicFunc.refGeneWithVer,AAChange.refGeneWithVer,avsnp147,avsnp150,esp6500siv2_all,1000g2015aug_all,ExAC_nonpsych_ALL,…,PVS1,PS1,PS2,PS3,PS4,PM1,PM2,PM3,PM4,PM5,PM6,PP1,PP2,PP3,PP4,PP5,BA1,BS1,BS2,BS3,BS4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,DN ID,Patient ID,Phenotype,Platform,Study,Pubmed ID,regsnp_fpr,regsnp_disease,regsnp_splicing_site
str,i64,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""chr1""",13273,""".""","""G""","""C""",1941.97,"""SNP_filter""","""AC=3;AF=0.5;AN=6;BaseQRankSum=…","""GT:AD:DP:GQ:PL""","""0/1:8,21:29:99:610,0,207""","""0/1:26,29:55:99:820,0,685""","""0/1:32,22:54:99:522,0,892""","""3""","""0.5""","""6""","""-1.439""","""138""","""3.9794""","""2.164""","""3""","""0.5""","""36.56""","""0.241""","""14.07""","""-0.54""","""0.924""","""2020-06-08""","""ncRNA_exonic""","""DDX11L1""",""".""",""".""",""".""","""rs531730856""","""rs531730856""",""".""","""0.0950479""",""".""",…,""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""","""."""
"""chr1""",13284,""".""","""G""","""A""",256.29,"""SNP_filter""","""AC=1;AF=0.167;AN=6;BaseQRankSu…","""GT:AD:DP:GQ:PL""","""0/0:32,0:32:93:0,93,1395""","""0/0:45,0:45:99:0,120,1800""","""0/1:42,13:55:99:265,0,1167""","""1""","""0.167""","""6""","""4.26""","""132""","""0""","""6.658""","""1""","""0.167""","""34.94""","""-1.109""","""4.66""","""1.54""","""1.004""","""2020-06-08""","""ncRNA_exonic""","""DDX11L1""",""".""",""".""",""".""","""rs548333521""","""rs548333521""",""".""","""0.00139776""",""".""",…,""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""","""."""
"""chr1""",13417,""".""","""C""","""CGAGA""",1211.93,"""PASS""","""AC=3;AF=0.5;AN=6;BaseQRankSum=…","""GT:AD:DP:GQ:PGT:PID:PL:PS""","""0/1:21,11:32:99:.:.:395,0,1238…","""0|1:24,11:35:99:0|1:13417_C_CG…","""0/1:35,12:47:99:.:.:440,0,2019…","""3""","""0.5""","""6""","""-0.933""","""125""","""3.9794""","""6.8""","""3""","""0.5""","""33.01""","""1.42""","""10.63""","""0.124""","""1.426""","""2020-06-08""","""ncRNA_exonic""","""DDX11L1""",""".""",""".""",""".""","""rs777038595""","""rs777038595""",""".""",""".""","""0.1797""",…,""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""","""."""
"""chr1""",13418,""".""","""G""","""A""",21.29,"""LowQD;SNP_filter;VeryLowQual""","""AC=1;AF=0.167;AN=6;BaseQRankSu…","""GT:AD:DP:GQ:PGT:PID:PL:PS""","""0/0:34,0:34:48:.:.:0,48,1076:.""","""0|1:33,5:38:30:1|0:13417_C_CGA…","""0/0:51,0:51:67:.:.:0,67,1759:.""","""1""","""0.167""","""6""","""1.76""","""123""","""0""","""15.651""","""1""","""0.167""","""33.24""","""-2.732""","""0.56""","""0.281""","""1.852""","""2020-06-08""","""ncRNA_exonic""","""DDX11L1""",""".""",""".""",""".""","""rs759389692""","""rs75175547""",""".""",""".""","""0""",…,""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""","

In [11]:

# Ajouter la colonne ID incrémentielle
import hashlib

# lf_parsed = lf_parsed.with_row_count(name="ROW_ID", offset=1)

# Fonction pour hash (utilisation via map_elements car polars ne supporte pas apply direct)
def hash_row(row):
    to_hash = f"{row['#CHROM']}_{row['POS']}_{row['REF']}_{row['ALT']}"
    return hashlib.sha256(to_hash.encode()).hexdigest()

lf_parsed = lf_parsed.with_columns(
    pl.struct(["#CHROM", "POS", "REF", "ALT"])
    .map_elements(lambda row: hash_row(row), return_dtype=pl.Utf8)
    .alias("VARIANT_HASH")
)

In [12]:
lf_parsed.limit(5).collect()

#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,1BD4I,1BD4M,1BD4P,AC,AF,AN,BaseQRankSum,DP,ExcessHet,FS,MLEAC,MLEAF,MQ,MQRankSum,QD,ReadPosRankSum,SOR,ANNOVAR_DATE,Func.refGeneWithVer,Gene.refGeneWithVer,GeneDetail.refGeneWithVer,ExonicFunc.refGeneWithVer,AAChange.refGeneWithVer,avsnp147,avsnp150,esp6500siv2_all,1000g2015aug_all,ExAC_nonpsych_ALL,…,PS1,PS2,PS3,PS4,PM1,PM2,PM3,PM4,PM5,PM6,PP1,PP2,PP3,PP4,PP5,BA1,BS1,BS2,BS3,BS4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,DN ID,Patient ID,Phenotype,Platform,Study,Pubmed ID,regsnp_fpr,regsnp_disease,regsnp_splicing_site,VARIANT_HASH
str,i64,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""chr1""",13273,""".""","""G""","""C""",1941.97,"""SNP_filter""","""AC=3;AF=0.5;AN=6;BaseQRankSum=…","""GT:AD:DP:GQ:PL""","""0/1:8,21:29:99:610,0,207""","""0/1:26,29:55:99:820,0,685""","""0/1:32,22:54:99:522,0,892""","""3""","""0.5""","""6""","""-1.439""","""138""","""3.9794""","""2.164""","""3""","""0.5""","""36.56""","""0.241""","""14.07""","""-0.54""","""0.924""","""2020-06-08""","""ncRNA_exonic""","""DDX11L1""",""".""",""".""",""".""","""rs531730856""","""rs531730856""",""".""","""0.0950479""",""".""",…,""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""","""03002d43bd273be064f9549c99dc5c…"
"""chr1""",13284,""".""","""G""","""A""",256.29,"""SNP_filter""","""AC=1;AF=0.167;AN=6;BaseQRankSu…","""GT:AD:DP:GQ:PL""","""0/0:32,0:32:93:0,93,1395""","""0/0:45,0:45:99:0,120,1800""","""0/1:42,13:55:99:265,0,1167""","""1""","""0.167""","""6""","""4.26""","""132""","""0""","""6.658""","""1""","""0.167""","""34.94""","""-1.109""","""4.66""","""1.54""","""1.004""","""2020-06-08""","""ncRNA_exonic""","""DDX11L1""",""".""",""".""",""".""","""rs548333521""","""rs548333521""",""".""","""0.00139776""",""".""",…,""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""","""f3fba87a45dc231d63adde9e0a2814…"
"""chr1""",13417,""".""","""C""","""CGAGA""",1211.93,"""PASS""","""AC=3;AF=0.5;AN=6;BaseQRankSum=…","""GT:AD:DP:GQ:PGT:PID:PL:PS""","""0/1:21,11:32:99:.:.:395,0,1238…","""0|1:24,11:35:99:0|1:13417_C_CG…","""0/1:35,12:47:99:.:.:440,0,2019…","""3""","""0.5""","""6""","""-0.933""","""125""","""3.9794""","""6.8""","""3""","""0.5""","""33.01""","""1.42""","""10.63""","""0.124""","""1.426""","""2020-06-08""","""ncRNA_exonic""","""DDX11L1""",""".""",""".""",""".""","""rs777038595""","""rs777038595""",""".""",""".""","""0.1797""",…,""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""",""".""","""fa35f50e416fc9592961fdd2dc6c43…"
"""chr1""",13418,""".""","""G""","""A""",21.29,"""LowQD;SNP_filter;VeryLowQual""","""AC=1;AF=0.167;AN=6;BaseQRankSu…","""GT:AD:DP:GQ:PGT:PID:PL:PS""","""0/0:34,0:34:48:.:.:0,48,1076:.""","""0|1:33,5:38:30:1|0:13417_C_CGA…","""0/0:51,0:51:67:.:.:0,67,1759:.""","""1""","""0.167""","""6""","""1.76""","""123""","""0""","""15.651""","""1""","""0.167""","""33.24""","""-2.732""","""0.56""","""0.281""","""1.852""","""2020-06-08""","""ncRNA_exonic""","""DDX11L1""",""".""",""".""",""".""","""rs759389692""","""rs75175547""",""".""",""".""","""0""",…,""".""",""".""",""".""",""".""",""".""",""".""",""".""","""."

In [13]:
print(lf_parsed.schema)


Schema([('#CHROM', String), ('POS', Int64), ('ID', String), ('REF', String), ('ALT', String), ('QUAL', Float64), ('FILTER', String), ('INFO', String), ('FORMAT', String), ('1BD4I', String), ('1BD4M', String), ('1BD4P', String), ('VARIANT_HASH', String)])


/tmp/ipykernel_642220/2506692253.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  print(lf_parsed.schema)


In [ ]:
lf_parsed.sink_parquet("../db/variants.parquet")

/tmp/ipykernel_642220/375505352.py:1: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  lf_parsed.sink_parquet("db/variants.parquet")
